In [49]:
import warnings
warnings.filterwarnings('ignore')

In [50]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

In [51]:
import sys
!{sys.executable} -m pip install statsmodels

In [52]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [53]:
 #import libraries
import pandas as pd
from sklearn.linear_model import LinearRegression

In [54]:
#define a function to calculate VIF
def vif(df):
  #initialize an empty dataframe to store VIF values
  vif_df = pd.DataFrame()
  #get the names of the features
  vif_df["feature"] = df.columns
  #loop through each feature and regress it against the others
  vif_df["VIF"] = [1 / (1 - LinearRegression(n_jobs=-1).fit(df.drop(feature, axis=1), df[feature]).score(df.drop(feature, axis=1), df[feature])) for feature in df.columns]
  #return the VIF dataframe
  return vif_df

In [55]:
asteroid_data = pd.read_csv('https://dagshub.com/mirsakhawathossain/Asteroid_ML/raw/cceba246d6e1755c40511f7ccbabd1ac819f0515/Data/sbdb_query_results.csv',low_memory=False).sort_index(axis=0)

In [56]:
asteroid_data.head(3)

,full_name,neo,pha,H,diameter,albedo,rot_per,e,a,q,i,om,w,ma,ad,n,per,moid_ld,class
0,1 Ceres (A801 AA),N,N,3.33,939.400,0.090,9.074170,0.0789,2.767,2.549,10.59,80.25,73.42,60.08,2.99,0.2141,1680.0,616.0,MBA
1,2 Pallas (A802 FA),N,N,4.12,513.000,0.155,7.813221,0.2302,2.770,2.132,34.93,172.92,310.87,40.60,3.41,0.2138,1680.0,478.0,MBA
2,3 Juno (A804 RA),N,N,5.16,246.596,0.214,7.210000,0.2562,2.669,1.985,12.99,169.84,247.74,37.02,3.35,0.2260,1590.0,403.0,MBA


In [57]:
asteroid_data.isnull().sum()

full_name          0
neo                4
pha            11255
H               1230
diameter     1172332
albedo       1173460
rot_per      1277860
e                  0
a                  0
q                  0
i                  0
om                 0
w                  0
ma                 1
ad                 4
n                  0
per                4
moid_ld        11254
class              0
dtype: int64

In [58]:
asteroid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311963 entries, 0 to 1311962
Data columns (total 19 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   full_name  1311963 non-null  object 
 1   neo        1311959 non-null  object 
 2   pha        1300708 non-null  object 
 3   H          1310733 non-null  float64
 4   diameter   139631 non-null   float64
 5   albedo     138503 non-null   float64
 6   rot_per    34103 non-null    float64
 7   e          1311963 non-null  float64
 8   a          1311963 non-null  float64
 9   q          1311963 non-null  float64
 10  i          1311963 non-null  float64
 11  om         1311963 non-null  float64
 12  w          1311963 non-null  float64
 13  ma         1311962 non-null  float64
 14  ad         1311959 non-null  float64
 15  n          1311963 non-null  float64
 16  per        1311959 non-null  float64
 17  moid_ld    1300709 non-null  float64
 18  class      1311963 non-null  object 
dtype

In [59]:
dataframe=asteroid_data.drop(columns=['neo','full_name'])

In [60]:
dataframe['diameter'].fillna(dataframe.groupby('class')['diameter'].transform('median'),inplace=True)
dataframe['H'].fillna(dataframe.groupby('class')['H'].transform('median'),inplace=True)
dataframe['albedo'].fillna(dataframe.groupby('class')['albedo'].transform('median'),inplace=True)
dataframe['ma'].fillna(dataframe.groupby('class')['ma'].transform('median'),inplace=True)
dataframe['ad'].fillna(dataframe.groupby('class')['ad'].transform('median'),inplace=True)
dataframe['per'].fillna(dataframe.groupby('class')['per'].transform('median'),inplace=True)
dataframe['moid_ld'].fillna(dataframe.groupby('class')['moid_ld'].transform('median'),inplace=True)
dataframe['rot_per'].fillna(dataframe.groupby('class')['rot_per'].transform('median'),inplace=True)

In [61]:
dataframe[['diameter','albedo','ad','per']]=dataframe[['diameter','albedo','ad','per']].fillna(value=dataframe[['diameter','albedo','ad','per']].median())

In [62]:
dataframe.isnull().sum()

pha         11255
H               0
diameter        0
albedo          0
rot_per         0
e               0
a               0
q               0
i               0
om              0
w               0
ma              0
ad              0
n               0
per             0
moid_ld         0
class           0
dtype: int64

In [63]:
dataframe = dataframe.drop(columns=['class','ad','om','a','ma','q','rot_per','w','albedo','n','pha'])

In [64]:
dataframe.isnull().sum()

H           0
diameter    0
e           0
i           0
per         0
moid_ld     0
dtype: int64

In [65]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming 'dataframe' is your DataFrame
# Drop any non-numeric columns or handle them appropriately for VIF calculation
numeric_df = dataframe.select_dtypes(include=['float64', 'int64'])

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = numeric_df.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_df.values, i) for i in range(numeric_df.shape[1])]

# Print the VIF dataframe
print(vif_data)


    Feature       VIF
0         H  6.155004
1  diameter  3.169350
2         e  4.338338
3         i  2.911504
4       per  1.001251
5   moid_ld  2.369248
